<a href="https://colab.research.google.com/github/nasthosh/kaggle_competition_python/blob/master/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
from google.colab import files
import numpy as np
import pandas as pd
import io
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [153]:
upload_data = files.upload()

Saving train.csv to train.csv


In [154]:
train_dataset = pd.read_csv(io.StringIO(upload_data['train.csv'].decode('utf-8')))
train_dataset.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [155]:
print(train_dataset)

         id keyword  ...                                               text target
0         1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1         4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2         5     NaN  ...  All residents asked to 'shelter in place' are ...      1
3         6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4         7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1
...     ...     ...  ...                                                ...    ...
7608  10869     NaN  ...  Two giant cranes holding a bridge collapse int...      1
7609  10870     NaN  ...  @aria_ahrary @TheTawniest The out of control w...      1
7610  10871     NaN  ...  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...      1
7611  10872     NaN  ...  Police investigating after an e-bike collided ...      1
7612  10873     NaN  ...  The Latest: More Homes Razed by Northern Calif...      1

[76

In [156]:
train_dataset.isna().sum(axis=0)

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [157]:
def remove_noise(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train_dataset.text=train_dataset.text.apply(lambda x: remove_noise(x))

In [158]:
train_dataset.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive wildfires evacuation orders in...,1
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1


In [159]:
def word_tokens(text):
  return word_tokenize(text)

train_dataset_tokens = train_dataset
train_dataset_tokens.text=train_dataset_tokens.text.apply(lambda x: word_tokens(x))

In [160]:
train_dataset_tokens.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,"[Our, Deeds, are, the, Reason, of, this, earth...",1
1,4,NaN,NaN,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,5,NaN,NaN,"[All, residents, asked, to, shelter, in, place...",1
3,6,NaN,NaN,"[people, receive, wildfires, evacuation, order...",1
4,7,NaN,NaN,"[Just, got, sent, this, photo, from, Ruby, Ala...",1


In [161]:
def remove_stopwords(text):
  return [word for word in text if not word in stopwords.words()]

train_remove_stopwords = train_dataset_tokens
train_remove_stopwords.text = train_remove_stopwords.text.apply(lambda x: remove_stopwords(x))

In [162]:
train_remove_stopwords.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,"[Our, Deeds, Reason, earthquake, May, ALLAH, F...",1
1,4,NaN,NaN,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,5,NaN,NaN,"[All, residents, asked, shelter, place, notifi...",1
3,6,NaN,NaN,"[people, receive, wildfires, evacuation, order...",1
4,7,NaN,NaN,"[Just, got, sent, photo, Ruby, Alaska, smoke, ...",1


In [148]:
bad_chars = ['#'] 
train_remove_unwanted_chars = train_remove_stopwords
train_remove_unwanted_chars.text = ''.join(str(i) for i in train_remove_unwanted_chars.text if not i in bad_chars)

In [149]:
train_remove_unwanted_chars.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,,1
1,4,NaN,NaN,,1
2,5,NaN,NaN,,1
3,6,NaN,NaN,,1
4,7,NaN,NaN,,1
